# परिवार संबंध ऑंटोलॉजी

यह उदाहरण [AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners) का हिस्सा है, और इसे [इस ब्लॉग पोस्ट](https://habr.com/post/270857/) से प्रेरित होकर बनाया गया है।

मुझे हमेशा परिवार में लोगों के बीच विभिन्न संबंधों को याद रखना मुश्किल लगता है। इस उदाहरण में, हम एक ऑंटोलॉजी लेंगे जो परिवार संबंधों को परिभाषित करती है, और वास्तविक वंशावली वृक्ष का उपयोग करेंगे, और फिर दिखाएंगे कि हम स्वचालित निष्कर्षण करके सभी रिश्तेदारों को कैसे खोज सकते हैं।

### वंशावली वृक्ष प्राप्त करना

उदाहरण के रूप में, हम [रोमानोव ज़ार परिवार](https://en.wikipedia.org/wiki/House_of_Romanov) का वंशावली वृक्ष लेंगे। परिवार संबंधों का वर्णन करने के लिए सबसे सामान्य प्रारूप [GEDCOM](https://en.wikipedia.org/wiki/GEDCOM) है। हम GEDCOM प्रारूप में रोमानोव परिवार का वंशावली वृक्ष लेंगे:


In [1]:
!head -15 data/tsars.ged

0 HEAD
1 CHAR UTF8
1 GEDC
2 VERS 5.5
0 @0@ INDI
1 NAME Mihail Fedorovich /Romanov/
1 SEX M
1 BIRT
2 DATE 1613
1 DEAT 
2 DATE 1645
1 FAMS @41@
0 @1@ INDI
1 NAME Evdokija Lukjanovna /Streshneva/
1 SEX F


GEDCOM फ़ाइल का उपयोग करने के लिए, हम `python-gedcom` लाइब्रेरी का उपयोग कर सकते हैं:


In [2]:
import sys
!{sys.executable} -m pip install python-gedcom

यह पुस्तकालय फ़ाइल पार्सिंग से संबंधित कुछ तकनीकी समस्याओं को दूर करता है, लेकिन यह अभी भी हमें पेड़ में सभी व्यक्तियों और परिवारों तक काफी निम्न-स्तरीय पहुंच प्रदान करता है। यहां बताया गया है कि हम फ़ाइल को कैसे पार्स कर सकते हैं और सभी व्यक्तियों की सूची दिखा सकते हैं:


In [3]:
from gedcom.parser import Parser
from gedcom.element.individual import IndividualElement
from gedcom.element.family import FamilyElement
g = Parser()
g.parse_file('data/tsars.ged')

In [4]:
d = g.get_element_dictionary()
[ (k,v.get_name()) for k,v in d.items() if isinstance(v,IndividualElement)]

[('@0@', ('Mihail Fedorovich', 'Romanov')),
 ('@1@', ('Evdokija Lukjanovna', 'Streshneva')),
 ('@2@', ('Aleksej Mihajlovich', 'Romanov')),
 ('@3@', ('Marija Ilinichna', 'Miloslavskaja')),
 ('@4@', ('Natalja Kirillovna', 'Naryshkina')),
 ('@5@', ('Marfa Matveevna', 'Apraksina')),
 ('@6@', ('Fedor Alekseevich', 'Romanov')),
 ('@7@', ('Sofja Aleksevna', 'Romanova')),
 ('@8@', ('Ivan V Alekseevich', 'Romanov')),
 ('@9@', ('Praskovja Fedorovna', 'Saltykova')),
 ('@10@', ('Ekaterina Ivanovna', 'Romanova')),
 ('@11@', ('Anna Ivanovna', 'Romanova')),
 ('@12@', ('Fridrih Vilgelm', 'Kurlandskij')),
 ('@13@', ('Karl Leopold', 'Meklenburg-Shverinskij')),
 ('@14@', ('Anna Leopoldovna', 'Meklenburg-Shverinskaja')),
 ('@15@', ('Anton Ulrih', 'Braunshvejg-Volfenbjuttelskij')),
 ('@16@', ('Ivan VI Antonovich', 'Braunshvejg-Volfenbjuttelskij')),
 ('@17@', ('Petr I Alekseevich', 'Romanov')),
 ('@18@', ('Evdokija Fedorovna', 'Lopuhina')),
 ('@19@', ('Ekaterina I Alekseevna', 'Mihajlova')),
 ('@20@', ('Ale

यहां बताया गया है कि हम परिवारों के बारे में जानकारी कैसे प्राप्त कर सकते हैं। ध्यान दें कि यह हमें **पहचानकर्ताओं** की एक सूची देता है, और यदि हमें अधिक स्पष्टता चाहिए तो हमें उन्हें नामों में बदलने की आवश्यकता है:


In [5]:
d = g.get_element_dictionary()
[ (k,[x.get_value() for x in v.get_child_elements()]) for k,v in d.items() if isinstance(v,FamilyElement)]

[('@41@', ['@0@', '@1@', '@2@']),
 ('@42@', ['@2@', '@3@', '@6@', '@7@', '@8@']),
 ('@43@', ['@8@', '@9@', '@10@', '@11@']),
 ('@44@', ['@13@', '@10@', '@14@']),
 ('@45@', ['@15@', '@14@', '@16@']),
 ('@46@', ['@2@', '@4@', '@17@']),
 ('@47@', ['@17@', '@18@', '@20@']),
 ('@48@', ['@20@', '@21@', '@22@']),
 ('@49@', ['@17@', '@19@', '@23@', '@24@']),
 ('@50@', ['@25@', '@23@', '@26@']),
 ('@51@', ['@26@', '@27@', '@28@']),
 ('@52@', ['@28@', '@30@', '@31@', '@33@']),
 ('@53@', ['@33@', '@34@', '@35@']),
 ('@54@', ['@35@', '@36@', '@37@']),
 ('@55@', ['@37@', '@38@', '@39@'])]

### परिवार ऑंटोलॉजी प्राप्त करना

अब, आइए [परिवार ऑंटोलॉजी](https://raw.githubusercontent.com/blokhin/genealogical-trees/master/data/header.ttl) पर नज़र डालें, जिसे सेमांटिक वेब ट्रिपलेट्स के सेट के रूप में परिभाषित किया गया है। इस ऑंटोलॉजी में `isUncleOf`, `isCousinOf` और कई अन्य संबंधों को परिभाषित किया गया है। ये सभी संबंध मूल प्रेडिकेट्स `isMotherOf`, `isFatherOf`, `isBrotherOf` और `isSisterOf` के आधार पर परिभाषित किए गए हैं। हम ऑंटोलॉजी का उपयोग करके स्वचालित तर्क के माध्यम से अन्य सभी संबंधों को निकालेंगे।

यहाँ `isAuntOf` प्रॉपर्टी की एक नमूना परिभाषा दी गई है, जिसे `isSisterOf` और `isParentOf` के संयोजन के रूप में परिभाषित किया गया है (*आंटी किसी के माता-पिता की बहन होती है*)।

```
fhkb:isAuntOf a owl:ObjectProperty ;
    rdfs:domain fhkb:Woman ;
    rdfs:range fhkb:Person ;
    owl:propertyChainAxiom ( fhkb:isSisterOf fhkb:isParentOf ) .
```


In [6]:
!head -20 data/onto.ttl

@prefix fhkb: <http://www.example.com/genealogy.owl#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://www.example.com/genealogy.owl#> a owl:Ontology .

fhkb:DomainEntity a owl:Class .

fhkb:Man a owl:Class ;
    owl:equivalentClass [ a owl:Class ;
            owl:intersectionOf ( fhkb:Person [ a owl:Restriction ;
                        owl:onProperty fhkb:hasSex ;
                        owl:someValuesFrom fhkb:Male ] ) ] .

fhkb:Woman a owl:Class ;
    owl:equivalentClass [ a owl:Class ;
            owl:intersectionOf ( fhkb:Person [ a owl:Restriction ;


### अनुमान के लिए ओंटोलॉजी बनाना

सरलता के लिए, हम एक ओंटोलॉजी फ़ाइल बनाएंगे जिसमें परिवार ओंटोलॉजी से मूल नियम और हमारे GEDCOM फ़ाइल से व्यक्तियों के बारे में तथ्य शामिल होंगे। हम GEDCOM फ़ाइल का विश्लेषण करेंगे, परिवारों और व्यक्तियों की जानकारी निकालेंगे, और उन्हें ट्रिपलेट्स में बदलेंगे।


In [7]:
!cp data/onto.ttl .

gedcom_dict = g.get_element_dictionary()
individuals, marriages = {}, {}

def term2id(el):
    return "i" + el.get_pointer().replace('@', '').lower()

out = open("onto.ttl","a")

for k, v in gedcom_dict.items():
    if isinstance(v,IndividualElement):
        children, siblings = set(), set()
        idx = term2id(v)

        title = v.get_name()[0] + " " + v.get_name()[1]
        title = title.replace('"', '').replace('[', '').replace(']', '').replace('(', '').replace(')', '').strip()

        own_families = g.get_families(v, 'FAMS')
        for fam in own_families:
            children |= set(term2id(i) for i in g.get_family_members(fam, "CHIL"))

        parent_families = g.get_families(v, 'FAMC')
        if len(parent_families):
            for member in g.get_family_members(parent_families[0], "CHIL"): # NB adoptive families i.e len(parent_families)>1 are not considered (TODO?)
                if member.get_pointer() == v.get_pointer():
                    continue
                siblings.add(term2id(member))

        if idx in individuals:
            children |= individuals[idx].get('children', set())
            siblings |= individuals[idx].get('siblings', set())
        individuals[idx] = {'sex': v.get_gender().lower(), 'children': children, 'siblings': siblings, 'title': title}

    elif isinstance(v,FamilyElement):
        wife, husb, children = None, None, set()
        children = set(term2id(i) for i in g.get_family_members(v, "CHIL"))

        try:
            wife = g.get_family_members(v, "WIFE")[0]
            wife = term2id(wife)
            if wife in individuals: individuals[wife]['children'] |= children
            else: individuals[wife] = {'children': children}
        except IndexError: pass
        try:
            husb = g.get_family_members(v, "HUSB")[0]
            husb = term2id(husb)
            if husb in individuals: individuals[husb]['children'] |= children
            else: individuals[husb] = {'children': children}
        except IndexError: pass

        if wife and husb: marriages[wife + husb] = (term2id(v), wife, husb)

for idx, val in individuals.items():
    added_terms = ''
    if val['sex'] == 'f':
        parent_predicate, sibl_predicate = "isMotherOf", "isSisterOf"
    else:
        parent_predicate, sibl_predicate = "isFatherOf", "isBrotherOf"
    if len(val['children']):
        added_terms += " ;\n    fhkb:" + parent_predicate + " " + ", ".join(["fhkb:" + i for i in val['children']])
    if len(val['siblings']):
        added_terms += " ;\n    fhkb:" + sibl_predicate + " " + ", ".join(["fhkb:" + i for i in val['siblings']])
    out.write("fhkb:%s a owl:NamedIndividual, owl:Thing%s ;\n    rdfs:label \"%s\" .\n" % (idx, added_terms, val['title']))

for k, v in marriages.items():
    out.write("fhkb:%s a owl:NamedIndividual, owl:Thing ;\n    fhkb:hasFemalePartner fhkb:%s ;\n    fhkb:hasMalePartner fhkb:%s .\n" % v)

out.write("[] a owl:AllDifferent ;\n    owl:distinctMembers (")
for idx in individuals.keys():
    out.write("    fhkb:" + idx)
for k, v in marriages.items():
    out.write("    fhkb:" + v[0])
out.write("    ) .")
out.close()

In [8]:
!tail onto.ttl

    fhkb:hasFemalePartner fhkb:i34 ;
    fhkb:hasMalePartner fhkb:i33 .
fhkb:i54 a owl:NamedIndividual, owl:Thing ;
    fhkb:hasFemalePartner fhkb:i36 ;
    fhkb:hasMalePartner fhkb:i35 .
fhkb:i55 a owl:NamedIndividual, owl:Thing ;
    fhkb:hasFemalePartner fhkb:i38 ;
    fhkb:hasMalePartner fhkb:i37 .
[] a owl:AllDifferent ;
    owl:distinctMembers (    fhkb:i0    fhkb:i1    fhkb:i2    fhkb:i3    fhkb:i4    fhkb:i5    fhkb:i6    fhkb:i7    fhkb:i8    fhkb:i9    fhkb:i10    fhkb:i11    fhkb:i12    fhkb:i13    fhkb:i14    fhkb:i15    fhkb:i16    fhkb:i17    fhkb:i18    fhkb:i19    fhkb:i20    fhkb:i21    fhkb:i22    fhkb:i23    fhkb:i24    fhkb:i25    fhkb:i26    fhkb:i27    fhkb:i28    fhkb:i29    fhkb:i30    fhkb:i31    fhkb:i32    fhkb:i33    fhkb:i34    fhkb:i35    fhkb:i36    fhkb:i37    fhkb:i38    fhkb:i39    fhkb:i40    fhkb:i41    fhkb:i42    fhkb:i43    fhkb:i44    fhkb:i45    fhkb:i46    fhkb:i47    fhkb:i48    fhkb:i49    fhkb:i50    fhkb:i51    fhkb:i52    fhkb:i53    fhkb:

### अनुमान लगाना 

अब हम इस ऑंटोलॉजी का उपयोग अनुमान लगाने और क्वेरी करने के लिए करना चाहते हैं। हम [RDFLib](https://github.com/RDFLib) का उपयोग करेंगे, जो RDF ग्राफ को विभिन्न प्रारूपों में पढ़ने, क्वेरी करने आदि के लिए एक लाइब्रेरी है। 

तार्किक अनुमान के लिए, हम [OWL-RL](https://github.com/RDFLib/OWL-RL) लाइब्रेरी का उपयोग करेंगे, जो हमें RDF ग्राफ का **Closure** बनाने की अनुमति देती है, यानी सभी संभावित अवधारणाओं और संबंधों को जोड़ना जो अनुमानित किए जा सकते हैं।


In [10]:
!{sys.executable} -m pip install rdflib
!{sys.executable} -m pip install git+https://github.com/RDFLib/OWL-RL.git

  Cloning https://github.com/RDFLib/OWL-RL.git to /tmp/pip-req-build-lbfzwi3m
  Running command git clone --filter=blob:none --quiet https://github.com/RDFLib/OWL-RL.git /tmp/pip-req-build-lbfzwi3m
  Resolved https://github.com/RDFLib/OWL-RL.git to commit a77e1791b88b54aace609bc6000aac14c7add4ff
  Preparing metadata (setup.py) ... done


चलो ओन्टोलॉजी फ़ाइल खोलते हैं और देखते हैं कि इसमें कितने ट्रिपलेट्स हैं:


In [11]:
import rdflib
from owlrl import DeductiveClosure, OWLRL_Extension

g = rdflib.Graph()
g.parse("onto.ttl", format="turtle")

print("Triplets found:%d" % len(g))

Triplets found:669


In [12]:
DeductiveClosure(OWLRL_Extension).expand(g)
print("Triplets after inference:%d" % len(g))

Triplets after inference:4246


### रिश्तेदारों के लिए क्वेरी करना

अब हम ग्राफ़ से क्वेरी कर सकते हैं ताकि लोगों के बीच विभिन्न संबंध देख सकें। हम **SPARQL** भाषा का उपयोग `query` मेथड के साथ कर सकते हैं। हमारे मामले में, चलिए हमारे परिवार वृक्ष में सभी **चाचा** देखते हैं:


In [13]:
qres = g.query(
    """SELECT DISTINCT ?aname ?bname
       WHERE {
          ?a fhkb:isUncleOf ?b .
          ?a rdfs:label ?aname .
          ?b rdfs:label ?bname .
       }""")

for row in qres:
    print("%s is uncle of %s" % row)

Fedor Alekseevich Romanov is uncle of Ekaterina Ivanovna Romanova
Aleksandr I Pavlovich Romanov is uncle of Aleksandr II Nikolaevich Romanov
Fedor Alekseevich Romanov is uncle of Anna Ivanovna Romanova


अलग-अलग पारिवारिक संबंधों के साथ प्रयोग करने में संकोच न करें। उदाहरण के लिए, आप `isAncestorOf` संबंध देख सकते हैं, जो किसी दिए गए व्यक्ति के सभी पूर्वजों को पुनरावृत्त रूप से परिभाषित करता है।

अंत में, चलिए सफाई करते हैं!


In [14]:
!rm onto.ttl


---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
